In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kaggle-wars-eclipse/sample_submission.csv
/kaggle/input/kaggle-wars-eclipse/test (2).csv
/kaggle/input/kaggle-wars-eclipse/train (1).csv


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

In [19]:
# Load the dataset
data = pd.read_csv('/kaggle/input/kaggle-wars-eclipse/train (1).csv')

# Split the dataset into training and testing sets



In [20]:
data_rating_5 = data[data['Rating'] == 5]
data_not_5 = data[data['Rating'] != 5]


In [21]:
fraction_to_keep = 0.35  # Change this value to the desired fraction of rows to keep

data_rating_5 = data_rating_5.sample(frac=fraction_to_keep, random_state=42)


In [22]:
data_balanced = pd.concat([data_rating_5, data_not_5])
data_balanced = data_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [23]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

data_balanced['cleaned_reviews'] = data_balanced['Review'].apply(preprocess_text)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
train_data, test_data = train_test_split(data_balanced, test_size=0.2, random_state=42)

In [25]:
# Define the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the reviews
train_encodings = tokenizer(train_data['Review'].tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_data['Review'].tolist(), truncation=True, padding=True, max_length=128)

# Create tf.data.Dataset
train_labels = train_data['Rating'].to_numpy() - 1  # 0-indexed labels
test_labels = test_data['Rating'].to_numpy() - 1

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(1000).batch(8)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(8)


In [26]:
# Initialize the model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Define the optimizer, loss function, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train the model
history = model.fit(train_dataset, epochs=5, validation_data=test_dataset)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
1296/1296 [==============================] - 267s 175ms/step - loss: 1.0648 - sparse_categorical_accuracy: 0.6104 - val_loss: 0.8370 - val_sparse_categorical_accuracy: 0.7141
Epoch 2/5
1296/1296 [==============================] - 217s 167ms/step - loss: 0.6395 - sparse_categorical_accuracy: 0.7807 - val_loss: 0.5688 - val_sparse_categorical_accuracy: 0.8067
Epoch 3/5
1296/1296 [==============================] - 215s 166ms/step - loss: 0.3471 - sparse_categorical_accuracy: 0.8880 - val_loss: 0.4372 - val_sparse_categorical_accuracy: 0.8688
Epoch 4/5
1296/1296 [==============================] - 217s 167ms/step - loss: 0.2440 - sparse_categorical_accuracy: 0.9187 - val_loss: 0.3848 - val_sparse_categorical_accuracy: 0.8843
Epoch 5/5
1296/1296 [==============================] - 215s 166ms/step - loss: 0.2018 - sparse_categorical_accuracy: 0.9307 - val_loss: 0.3795 - val_sparse_categorical_accuracy: 0.8978


In [27]:
data1 = pd.read_csv('/kaggle/input/kaggle-wars-eclipse/test (2).csv')

In [29]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

data1['cleaned_reviews'] = data1['Review'].apply(preprocess_text)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
data1

,Review,cleaned_reviews
0,This used to be my favorite game and it is sti...,used favorite game still
1,Every single time i put the online multi-playe...,every single time put online multi player mode...
2,Determined to make us watch ads! Wouldn't be a...,determined make us watch ads issue choose get ...
3,Super great running game,super great running game
4,"Hello NetEase, I really loved the game it dese...",hello netease really loved game deserve downlo...
...,...,...
6995,This is like playing real football,like playing real football
6996,Please fix aim accuracy,please fix aim accuracy
6997,"Fix your game ranking and chat, this game suck...",fix game ranking chat game sucks communication
6998,give coins 500000,give coins 500000


In [28]:
def predict_batch(model, tokenized_texts, batch_size=32):
    num_samples = len(tokenized_texts['input_ids'])
    logits_list = []

    for i in range(0, num_samples, batch_size):
        batch = {
            key: value[i:i + batch_size]
            for key, value in tokenized_texts.items()
        }

        batch_logits = model(batch).logits
        logits_list.append(batch_logits)

    logits = np.vstack(logits_list)
    return logits

# Predict on the test dataset
test_reviews = data1['cleaned_reviews'].tolist()
test_tokens = tokenizer(test_reviews, truncation=True, padding=True, max_length=128, return_tensors='tf')
logits = predict_batch(model, test_tokens, batch_size=32)
predictions = np.argmax(logits, axis=-1) + 1  # Convert back to 1-indexed labels

# Save the predictions as a CSV file
output = pd.DataFrame({ 'Predicted_Rating': predictions})
output.to_csv('predictions.csv', index=False)
